In [1]:
!pip install pytorchvideo
!pip install torchsummary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 972.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 28.0 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188714 sha256=f4c272a770f0d0959941438cbc6f54ebac46ef5fa6d962d039e89fa202ba5907
  Stored in directory: /root/.cache/pip/wheels/e8/51/05/053b29bac2400cbbae2fb7cfc41afd280d627bca7c9363ca80
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=9fad89680b7d9fb5d1e95c8522e15b77261290248e27b1c10e9b17e2983204f7
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built pytorchvideo fvcore


In [2]:
import torch
import numpy as np
from torch.utils.data import (
    Dataset,
    DataLoader,
) 
import pickle


import os

import math
import torch.nn as nn
import torch.nn.functional as F

from torchvision.transforms import Compose, Lambda, Grayscale,Normalize, CenterCrop,Resize
#from torchvision.transforms._transforms_video import CenterCropVideo
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    UniformTemporalSubsample,
)

from tqdm import tqdm

from collections import OrderedDict

import torch.optim as optim

from torch.autograd import Variable


import torchvision.models as models

import sys
from torchsummary import summary






class SignDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y


    def __len__(self):

        return len(self.y)

        # length = 0
        # with open(self.file, 'rb') as f:
        #     data = pickle.load(f)

        # for x in data:
        #     length += len(data[x])
        # return length


    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

In [3]:
IMAGE_HEIGHT = 720
IMAGE_WIDTH = 800
IMAGE_CHANNEL = 1
NUM_FRAMES = 25
NUM_CLASSES = 60







def transform_data(x):
    
    transform =  ApplyTransformToKey(
        key="video",
        transform=Compose(
            [
                Lambda(lambda x: x/255.0),
                Normalize(([0.2933, 0.4799, 0.6723]), ([0.1576, 0.1674, 0.2477])),
                Grayscale(num_output_channels=1),
                CenterCrop([720,900]),
                Resize([512,512]),
                Lambda(lambda x: x.permute(1,2,3,0)),#(channel, frames(depth), height, width)
#                 Lambda(lambda x: print(x.shape))

            ]

        ),
    )
    
    return transform(x)

def lp_video(video,start_time, end_time):
    global means,stds
    global nst
    
    video_data = video.get_clip(start_sec=float(start_time)/1000.0, end_sec=float(end_time)/1000.0)
            #print(video_data["video"].shape)
 
    if video_data["video"] is None:
        return None        # or pass
    else:
    
        #video_data["video"] = Grayscale(num_output_channels=1)((video_data["video"]).permute(1,0,2,3))
    #             video_data["video"] = video_data["video"]/255
                #print(video_data["video"].shape)
        video_data["video"] = video_data["video"].permute(1,0,2,3)
        
#         std, mean = torch.std_mean(video_data["video"],dim=[0,2,3])
#         std = std/255.0
#         #print(std)
#         mean = mean/255.0
#         #print(mean)
#         means += mean
#         stds +=std
#         nst += 1
        video_data = transform_data( video_data)

        return video_data["video"]

def load_dataloaders(batch_size,start,end):

    
    train_inputs =[] #x
    train_classes = [] #y
    global neg

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        time_start = []
        time_end = []
        filename = key
        print("file",filename)
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions
        
        for x in data[key]:
            img_cls = x[0]
            time_start.append(x[1])
            time_end.append(x[2])
            
            
            # start_time = x[1]
            # end_time = x[2]
            vid = lp_video(video,x[1], x[2])

            for m in torch.unbind(vid, dim=3):
                train_classes.append(img_cls)
                train_inputs.append(m)


            #some negative classes too
        for i in range(len(time_start)-2):
            if (time_start[i+1]- time_end[i])>1000.0:
                start_t = time_end[i]+800.0
                end_t = time_end[i]+880.0
                vid = lp_video(video,start_t, end_t)
                if vid is None:
                    break
                else:
                    for m in torch.unbind(vid, dim=3):
                        train_classes.append(60)
                        train_inputs.append(m)
            
        
            
    signds = SignDataset(train_inputs, train_classes)
    trainlen = int(len(signds)*0.8)
    torch.manual_seed(0)
    
    train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
    trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

    valloader = DataLoader(val_test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  
    return trainloader, valloader
        
    

In [4]:
def load_testloader(batch_size,start,end):

    time_start = []
    time_end = []
    train_inputs =[] #x
    train_classes = [] #y
    

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        filename = key
        print("file",filename)
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions

        for x in data[key]:
            img_cls = x[0]
            time_start.append(x[1])
            time_end.append(x[2])
            
            
            # start_time = x[1]
            # end_time = x[2]
            vid = lp_video(video,x[1], x[2])

            for m in torch.unbind(vid, dim=3):
                train_classes.append(img_cls)
                train_inputs.append(m)


            #some negative classes too
        for i in range(0,len(time_start)//2):
            if (time_start[i+1]- time_end[i])>1000.0:
                start_t = time_end[i]+700.0
                end_t = time_end[i]+780.0
                vid = lp_video(video,start_t, end_t)
                if vid is None:
                    break
                else:
                    for m in torch.unbind(vid, dim=3):
                        train_classes.append(60)
                        train_inputs.append(m)

            
        
            

    signds = SignDataset(train_inputs, train_classes)
    
    

    testloader = DataLoader(signds, batch_size=batch_size, shuffle=True, num_workers=2)

    return testloader

In [5]:
def load_vid(batch_size,start,end):


    train_inputs =[] #x

    video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Val_Set/VALIDATION/MSSL_VAL_SET_VIDEOS/p01_n131.mp4")
    # file functions

    vid = lp_video(video,start, end)

    for m in torch.unbind(vid, dim=3):
        train_inputs.append(m)

    

    testloader = DataLoader(train_inputs, batch_size=batch_size, shuffle=True, num_workers=2)

    return testloader

In [6]:
"""
Creates a EfficientNetV2 Model as defined in:
Mingxing Tan, Quoc V. Le. (2021). 
EfficientNetV2: Smaller Models and Faster Training
arXiv preprint arXiv:2104.00298.
import from https://github.com/d-li14/mobilenetv2.pytorch
"""



__all__ = ['effnetv2_s', 'effnetv2_m', 'effnetv2_l', 'effnetv2_xl']


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


# SiLU (Swish) activation function
if hasattr(nn, 'SiLU'):
    SiLU = nn.SiLU
else:
    # For compatibility with old PyTorch versions
    class SiLU(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)

 
class SELayer(nn.Module):
    def __init__(self, inp, oup, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(oup, _make_divisible(inp // reduction, 8)),
                SiLU(),
                nn.Linear(_make_divisible(inp // reduction, 8), oup),
                nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


class MBConv(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, use_se):
        super(MBConv, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup
        if use_se:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                SELayer(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # fused
                nn.Conv2d(inp, hidden_dim, 3, stride, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )


    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class EffNetV2(nn.Module):
    def __init__(self, cfgs, num_classes=61, width_mult=1.):
        super(EffNetV2, self).__init__()
        self.cfgs = cfgs

        # building first layer
        input_channel = _make_divisible(24 * width_mult, 8)
        layers = [conv_3x3_bn(1, input_channel, 2)]
        # building inverted residual blocks
        block = MBConv
        for t, c, n, s, use_se in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t, use_se))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1792 * width_mult, 8) if width_mult > 1.0 else 1792
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.001)
                m.bias.data.zero_()


def effnetv2_s(**kwargs):
    """
    Constructs a EfficientNetV2-S model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  2, 1, 0],
        [4,  48,  4, 2, 0],
        [4,  64,  4, 2, 0],
        [4, 128,  6, 2, 1],
        [6, 160,  9, 1, 1],
        [6, 256, 15, 2, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_m(**kwargs):
    """
    Constructs a EfficientNetV2-M model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  3, 1, 0],
        [4,  48,  5, 2, 0],
        [4,  80,  5, 2, 0],
        [4, 160,  7, 2, 1],
        [6, 176, 14, 1, 1],
        [6, 304, 18, 2, 1],
        [6, 512,  5, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_l(**kwargs):
    """
    Constructs a EfficientNetV2-L model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  7, 2, 0],
        [4,  96,  7, 2, 0],
        [4, 192, 10, 2, 1],
        [6, 224, 19, 1, 1],
        [6, 384, 25, 2, 1],
        [6, 640,  7, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_xl(**kwargs):
    """
    Constructs a EfficientNetV2-XL model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  8, 2, 0],
        [4,  96,  8, 2, 0],
        [4, 192, 16, 2, 1],
        [6, 256, 24, 1, 1],
        [6, 512, 32, 2, 1],
        [6, 640,  8, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)

In [7]:

def evluate(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    
        
    for batch_idx, data in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            

            output = model(data.half())  
            
            _,pred = output.max(1)# get the index of the max log-probability
            print(pred)



In [8]:

def test(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    
        
    for batch_idx, (data, target) in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            target = (target).to(device)

            output = model(data.half())  
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            #print("correct", correct)

    acc = 100.0 * correct / num_samples
    
    print('{} Accuracy: {}/{} ({:.0f}%)'.format(
                mode,correct, num_samples,
                100. * correct / num_samples, acc))

In [9]:

    


def train(model, device, train_loader,validloader,scheduler, optimizer,criterion,epochs,lr):
    print("Train start")
    breakout = False
    model.half()
    model.cuda()

    
    model.train()

    optimizer.param_groups[0]['lr'] = lr
    
    for epoch in range(epochs):
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            
        if optimizer.param_groups[0]['lr'] != 1e-3:    
            scheduler.step()
        #print("lr:",optimizer.param_groups[0]['lr'])
        
        
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 95:
            
            print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            
            test(model,device,validloader, mode = "Validating")
#             test(model, device, testloader, mode = "Test before Training on validation set")
           # model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epochs)
#             test(model, device, testloader, mode = "Test after training on validation set")
            
            
            
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer34.pt")
            
            breakout = True
            print("breakout")
            return model, optimizer
            
            
            
       
        

        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
        if (((epoch+1)%5) == 0) :
            test(model,device,validloader)
            model.train()
            
    if breakout:
            print("breakout")
            return
    
    
    
    #model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epochs)
    test(model, device, validloader, mode= "test set ")
    
    
        
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer34.pt")
    return model, optimizer
    
    
        


In [10]:
n_classes = 61


model = nn.Sequential(OrderedDict([
    ('frontend',effnetv2_m())
]))





# specify loss function (categorical cross-entropy)

# specify optimizer and learning rate
optimizer = optim.SGD(
    [
        
        {"params": model.frontend.parameters(), "lr": 1e-2},
  ],
  momentum = 0.9
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


samples_per_cls = [166.0, 370.0, 63.0, 232.0, 905.0, 102.0, 537.0, 993.0, 133.0, 154.0, 286.0, 435.0, 452.0, 779.0, 354.0, 136.0, 232.0, 2154.0, 374.0, 787.0, 337.0, 268.0, 954.0, 167.0, 543.0, 324.0, 669.0, 333.0, 211.0, 433.0, 140.0, 665.0, 107.0, 305.0, 162.0, 61.0, 1229.0, 958.0, 897.0, 543.0, 625.0, 334.0, 776.0, 111.0, 412.0, 583.0, 255.0, 103.0, 108.0, 201.0, 384.0, 632.0, 13.0, 322.0, 664.0, 435.0, 194.0, 322.0, 113.0, 462.0,2330.0]
no_of_classes=61
beta=0.9999
effective_num = 1.0 - np.power(beta, samples_per_cls)
weights = (1.0 - beta) / np.array(effective_num)
weights = weights / np.sum(weights) * no_of_classes
    

weights = torch.FloatTensor(weights).cuda()

criterion = nn.CrossEntropyLoss(weight = weights.half())

# state = torch.load("../input/fmosel/model_optimizer34.pt")
# model.load_state_dict(state['model_state_dict'])
# model.half()
# model.cuda()
# optimizer.load_state_dict(state['optimizer_state_dict'])

In [11]:
#print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

In [12]:
#summary(model, (1, 512, 640), device='cpu')#1334+986 =2330

In [13]:
trainloader, valloader = load_dataloaders(batch_size=16,start=0, end=6)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p01_n002
file p06_n019
file p06_n007
file p01_n053
file p04_n074
file p05_n077
Train start
Epoch: 1 , Training Accuracy: 90/1368 (7%) Training Loss: 0.216059
Epoch: 2 , Training Accuracy: 219/1368 (16%) Training Loss: 0.176734
Epoch: 3 , Training Accuracy: 389/1368 (28%) Training Loss: 0.137640
Epoch: 4 , Training Accuracy: 590/1368 (43%) Training Loss: 0.100729
Epoch: 5 , Training Accuracy: 753/1368 (55%) Training Loss: 0.070814
Validate Accuracy: 225/342 (66%)
Epoch: 6 , Training Accuracy: 993/1368 (73%) Training Loss: 0.039461
Epoch: 7 , Training Accuracy: 1069/1368 (78%) Training Loss: 0.030265
Epoch: 8 , Training Accuracy: 1097/1368 (80%) Training Loss: 0.026047
Epoch: 9 , Training Accuracy: 1120/1368 (82%) Training Loss: 0.024520
Epoch: 10 , Training Accuracy: 1156/1368 (85%) Training Loss: 0.022081
Validate Accuracy: 280/342 (82%)
Epoch: 11 , Training Accuracy: 1173/1368 (86%) Training Loss: 0.018924
Epoch: 12 , Training Accuracy: 1200/1368 (88%) Training Loss: 0.017100
Epo

In [14]:
trainloader, valloader = load_dataloaders(batch_size=16,start=6, end=12)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p05_n027
file p05_n022
file p01_n110
file p06_n036
file p01_n069
file p06_n109
Train start
Epoch: 1 , Training Accuracy: 300/1422 (21%) Training Loss: 0.207919
Epoch: 2 , Training Accuracy: 639/1422 (45%) Training Loss: 0.095690
Epoch: 3 , Training Accuracy: 871/1422 (61%) Training Loss: 0.056393
Epoch: 4 , Training Accuracy: 1021/1422 (72%) Training Loss: 0.037834
Epoch: 5 , Training Accuracy: 1128/1422 (79%) Training Loss: 0.027530
Validate Accuracy: 191/356 (54%)
Epoch: 6 , Training Accuracy: 1181/1422 (83%) Training Loss: 0.019137
Epoch: 7 , Training Accuracy: 1237/1422 (87%) Training Loss: 0.013076
Epoch: 8 , Training Accuracy: 1281/1422 (90%) Training Loss: 0.010233
Epoch: 9 , Training Accuracy: 1285/1422 (90%) Training Loss: 0.009426
Epoch: 10 , Training Accuracy: 1287/1422 (91%) Training Loss: 0.009313
Validate Accuracy: 303/356 (85%)
Epoch: 11 , Training Accuracy: 1300/1422 (91%) Training Loss: 0.008324
Epoch: 12 , Training Accuracy: 1301/1422 (91%) Training Loss: 0.00801

In [15]:
trainloader, valloader = load_dataloaders(batch_size=16,start=12, end=18)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p04_n089
file p01_n044
file p05_n037
file p04_n038
file p06_n055
file p01_n099
Train start
Epoch: 1 , Training Accuracy: 559/1380 (41%) Training Loss: 0.154587
Epoch: 2 , Training Accuracy: 970/1380 (70%) Training Loss: 0.051636
Epoch: 3 , Training Accuracy: 1097/1380 (79%) Training Loss: 0.029317
Epoch: 4 , Training Accuracy: 1148/1380 (83%) Training Loss: 0.023647
Epoch: 5 , Training Accuracy: 1239/1380 (90%) Training Loss: 0.011124
Validate Accuracy: 325/346 (94%)
Epoch: 6 , Training Accuracy: 1313/1380 (95%) Training Loss: 0.006418
Validating Accuracy: 326/346 (94%)
breakout


In [16]:
trainloader, valloader = load_dataloaders(batch_size=16,start=18, end=24)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p02_n066
file p04_n035
file p01_n057
file p01_n105
file p04_n058
file p05_n043
Train start
Epoch: 1 , Training Accuracy: 401/1102 (36%) Training Loss: 0.255567
Epoch: 2 , Training Accuracy: 686/1102 (62%) Training Loss: 0.113121
Epoch: 3 , Training Accuracy: 810/1102 (74%) Training Loss: 0.071901
Epoch: 4 , Training Accuracy: 905/1102 (82%) Training Loss: 0.047044
Epoch: 5 , Training Accuracy: 939/1102 (85%) Training Loss: 0.033755
Validate Accuracy: 242/276 (88%)
Epoch: 6 , Training Accuracy: 973/1102 (88%) Training Loss: 0.026554
Epoch: 7 , Training Accuracy: 985/1102 (89%) Training Loss: 0.019999
Epoch: 8 , Training Accuracy: 1010/1102 (92%) Training Loss: 0.014649
Epoch: 9 , Training Accuracy: 1025/1102 (93%) Training Loss: 0.011661
Epoch: 10 , Training Accuracy: 1027/1102 (93%) Training Loss: 0.010376
Validate Accuracy: 247/276 (89%)
Epoch: 11 , Training Accuracy: 1028/1102 (93%) Training Loss: 0.008684
Epoch: 12 , Training Accuracy: 1034/1102 (94%) Training Loss: 0.008193
Ep

In [17]:
trainloader, valloader = load_dataloaders(batch_size=16,start=24, end=30)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p06_n020
file p02_n001
file p04_n032
file p05_n084
file p01_n102
file p06_n096
Train start
Epoch: 1 , Training Accuracy: 553/1338 (41%) Training Loss: 0.148285
Epoch: 2 , Training Accuracy: 944/1338 (71%) Training Loss: 0.047304
Epoch: 3 , Training Accuracy: 1106/1338 (83%) Training Loss: 0.025739
Epoch: 4 , Training Accuracy: 1188/1338 (89%) Training Loss: 0.012488
Epoch: 5 , Training Accuracy: 1253/1338 (94%) Training Loss: 0.007135
Validate Accuracy: 306/335 (91%)
Epoch: 6 , Training Accuracy: 1303/1338 (97%) Training Loss: 0.003400
Validating Accuracy: 309/335 (92%)
breakout


In [18]:
trainloader, valloader = load_dataloaders(batch_size=16,start=30, end=36)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p02_n104
file p01_n090
file p01_n040
file p04_n073
file p01_n067
file p05_n004
Train start
Epoch: 1 , Training Accuracy: 516/1277 (40%) Training Loss: 0.151150
Epoch: 2 , Training Accuracy: 973/1277 (76%) Training Loss: 0.037240
Epoch: 3 , Training Accuracy: 1108/1277 (87%) Training Loss: 0.017337
Epoch: 4 , Training Accuracy: 1197/1277 (94%) Training Loss: 0.006516
Epoch: 5 , Training Accuracy: 1241/1277 (97%) Training Loss: 0.003590
Validating Accuracy: 295/320 (92%)
breakout


In [19]:
trainloader, valloader = load_dataloaders(batch_size=16,start=30, end=36)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p02_n104
file p01_n090
file p01_n040
file p04_n073
file p01_n067
file p05_n004
Train start
Epoch: 1 , Training Accuracy: 1269/1277 (99%) Training Loss: 0.002041
Validating Accuracy: 294/320 (92%)
breakout


In [20]:
trainloader, valloader = load_dataloaders(batch_size=16,start=36, end=42)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p06_n061
file p05_n092
file p06_n015
file p04_n106
file p05_n095
file p02_n072
Train start
Epoch: 1 , Training Accuracy: 794/1440 (55%) Training Loss: 0.101476
Epoch: 2 , Training Accuracy: 1163/1440 (81%) Training Loss: 0.028264
Epoch: 3 , Training Accuracy: 1306/1440 (91%) Training Loss: 0.010532
Epoch: 4 , Training Accuracy: 1364/1440 (95%) Training Loss: 0.005840
Epoch: 5 , Training Accuracy: 1408/1440 (98%) Training Loss: 0.002668
Validating Accuracy: 332/360 (92%)
breakout


In [21]:
trainloader, valloader = load_dataloaders(batch_size=16,start=42, end=48)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p05_n031
file p01_n060
file p05_n048
file p05_n029
file p01_n000
file p01_n050
Train start
Epoch: 1 , Training Accuracy: 582/1220 (48%) Training Loss: 0.135600
Epoch: 2 , Training Accuracy: 961/1220 (79%) Training Loss: 0.030862
Epoch: 3 , Training Accuracy: 1088/1220 (89%) Training Loss: 0.012970
Epoch: 4 , Training Accuracy: 1116/1220 (91%) Training Loss: 0.010160
Epoch: 5 , Training Accuracy: 1119/1220 (92%) Training Loss: 0.008678
Validate Accuracy: 266/305 (87%)
Epoch: 6 , Training Accuracy: 1121/1220 (92%) Training Loss: 0.008969
Epoch: 7 , Training Accuracy: 1131/1220 (93%) Training Loss: 0.007777
Epoch: 8 , Training Accuracy: 1127/1220 (92%) Training Loss: 0.008087
Epoch: 9 , Training Accuracy: 1137/1220 (93%) Training Loss: 0.007538
Epoch: 10 , Training Accuracy: 1137/1220 (93%) Training Loss: 0.006383
Validate Accuracy: 273/305 (90%)
Epoch: 11 , Training Accuracy: 1134/1220 (93%) Training Loss: 0.007317
Epoch: 12 , Training Accuracy: 1140/1220 (93%) Training Loss: 0.0064

In [22]:
trainloader, valloader = load_dataloaders(batch_size=16,start=48, end=54)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p05_n025
file p04_n011
file p01_n023
file p01_n042
file p01_n111
file p05_n083
Train start
Epoch: 1 , Training Accuracy: 490/1201 (41%) Training Loss: 0.186810
Epoch: 2 , Training Accuracy: 792/1201 (66%) Training Loss: 0.078810
Epoch: 3 , Training Accuracy: 889/1201 (74%) Training Loss: 0.053428
Epoch: 4 , Training Accuracy: 944/1201 (79%) Training Loss: 0.041053
Epoch: 5 , Training Accuracy: 988/1201 (82%) Training Loss: 0.035374
Validate Accuracy: 252/301 (84%)
Epoch: 6 , Training Accuracy: 1015/1201 (85%) Training Loss: 0.029991
Epoch: 7 , Training Accuracy: 1025/1201 (85%) Training Loss: 0.025218
Epoch: 8 , Training Accuracy: 1050/1201 (87%) Training Loss: 0.020846
Epoch: 9 , Training Accuracy: 1071/1201 (89%) Training Loss: 0.017118
Epoch: 10 , Training Accuracy: 1077/1201 (90%) Training Loss: 0.013983
Validate Accuracy: 264/301 (88%)
Epoch: 11 , Training Accuracy: 1096/1201 (91%) Training Loss: 0.016923
Epoch: 12 , Training Accuracy: 1104/1201 (92%) Training Loss: 0.010744


In [23]:
trainloader, valloader = load_dataloaders(batch_size=16,start=54, end=60)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p05_n094
file p04_n103
file p01_n093
file p04_n003
file p05_n078
file p01_n052
Train start
Epoch: 1 , Training Accuracy: 497/800 (62%) Training Loss: 0.084719
Epoch: 2 , Training Accuracy: 700/800 (88%) Training Loss: 0.015254
Epoch: 3 , Training Accuracy: 747/800 (93%) Training Loss: 0.008330
Epoch: 4 , Training Accuracy: 744/800 (93%) Training Loss: 0.006802
Epoch: 5 , Training Accuracy: 748/800 (94%) Training Loss: 0.006235
Validate Accuracy: 184/200 (92%)
Epoch: 6 , Training Accuracy: 752/800 (94%) Training Loss: 0.005752
Epoch: 7 , Training Accuracy: 755/800 (94%) Training Loss: 0.004985
Epoch: 8 , Training Accuracy: 756/800 (94%) Training Loss: 0.005158
Epoch: 9 , Training Accuracy: 752/800 (94%) Training Loss: 0.005148
Epoch: 10 , Training Accuracy: 761/800 (95%) Training Loss: 0.004426
Validating Accuracy: 187/200 (94%)
breakout


In [24]:
trainloader, valloader = load_dataloaders(batch_size=16,start=60, end=66)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p02_n098
file p06_n013
file p05_n006
file p01_n063
file p02_n097
file p05_n065
Train start
Epoch: 1 , Training Accuracy: 667/1426 (47%) Training Loss: 0.121744
Epoch: 2 , Training Accuracy: 1130/1426 (79%) Training Loss: 0.028184
Epoch: 3 , Training Accuracy: 1250/1426 (88%) Training Loss: 0.013311
Epoch: 4 , Training Accuracy: 1280/1426 (90%) Training Loss: 0.010402
Epoch: 5 , Training Accuracy: 1287/1426 (90%) Training Loss: 0.010684
Validate Accuracy: 307/357 (86%)
Epoch: 6 , Training Accuracy: 1288/1426 (90%) Training Loss: 0.009905
Epoch: 7 , Training Accuracy: 1287/1426 (90%) Training Loss: 0.009724
Epoch: 8 , Training Accuracy: 1294/1426 (91%) Training Loss: 0.011507
Epoch: 9 , Training Accuracy: 1301/1426 (91%) Training Loss: 0.009282
Epoch: 10 , Training Accuracy: 1309/1426 (92%) Training Loss: 0.008264
Validate Accuracy: 308/357 (86%)
Epoch: 11 , Training Accuracy: 1301/1426 (91%) Training Loss: 0.009015
Epoch: 12 , Training Accuracy: 1317/1426 (92%) Training Loss: 0.007

In [25]:
trainloader, valloader = load_dataloaders(batch_size=16,start=66, end=72)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p01_n108
file p06_n018
file p05_n009
file p05_n071
file p04_n091
file p05_n075
Train start
Epoch: 1 , Training Accuracy: 487/828 (59%) Training Loss: 0.111180
Epoch: 2 , Training Accuracy: 732/828 (88%) Training Loss: 0.017254
Epoch: 3 , Training Accuracy: 768/828 (93%) Training Loss: 0.007670
Epoch: 4 , Training Accuracy: 786/828 (95%) Training Loss: 0.006367
Epoch: 5 , Training Accuracy: 784/828 (95%) Training Loss: 0.005526
Validate Accuracy: 190/208 (91%)
Epoch: 6 , Training Accuracy: 790/828 (95%) Training Loss: 0.005555
Validating Accuracy: 189/208 (91%)
breakout


In [26]:
trainloader, valloader = load_dataloaders(batch_size=16,start=72, end=78)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p01_n012
file p05_n076
file p04_n005
file p04_n046
file p01_n045
file p05_n041
Train start
Epoch: 1 , Training Accuracy: 514/1296 (40%) Training Loss: 0.190071
Epoch: 2 , Training Accuracy: 864/1296 (67%) Training Loss: 0.074351
Epoch: 3 , Training Accuracy: 963/1296 (74%) Training Loss: 0.049773
Epoch: 4 , Training Accuracy: 1024/1296 (79%) Training Loss: 0.036311
Epoch: 5 , Training Accuracy: 1066/1296 (82%) Training Loss: 0.030289
Validate Accuracy: 263/324 (81%)
Epoch: 6 , Training Accuracy: 1093/1296 (84%) Training Loss: 0.024681
Epoch: 7 , Training Accuracy: 1128/1296 (87%) Training Loss: 0.020268
Epoch: 8 , Training Accuracy: 1141/1296 (88%) Training Loss: 0.018751
Epoch: 9 , Training Accuracy: 1148/1296 (89%) Training Loss: 0.016282
Epoch: 10 , Training Accuracy: 1174/1296 (91%) Training Loss: 0.014055
Validate Accuracy: 277/324 (85%)
Epoch: 11 , Training Accuracy: 1180/1296 (91%) Training Loss: 0.013012
Epoch: 12 , Training Accuracy: 1193/1296 (92%) Training Loss: 0.01223

In [27]:
trainloader, valloader = load_dataloaders(batch_size=16,start=78, end=84)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p01_n008
file p02_n101
file p01_n085
file p05_n064
file p05_n017
file p01_n028
Train start
Epoch: 1 , Training Accuracy: 615/1332 (46%) Training Loss: 0.146038
Epoch: 2 , Training Accuracy: 965/1332 (72%) Training Loss: 0.051996
Epoch: 3 , Training Accuracy: 1071/1332 (80%) Training Loss: 0.031580
Epoch: 4 , Training Accuracy: 1137/1332 (85%) Training Loss: 0.024866
Epoch: 5 , Training Accuracy: 1175/1332 (88%) Training Loss: 0.017638
Validate Accuracy: 272/334 (81%)
Epoch: 6 , Training Accuracy: 1196/1332 (90%) Training Loss: 0.016112
Epoch: 7 , Training Accuracy: 1219/1332 (92%) Training Loss: 0.013074
Epoch: 8 , Training Accuracy: 1224/1332 (92%) Training Loss: 0.010617
Epoch: 9 , Training Accuracy: 1238/1332 (93%) Training Loss: 0.008990
Epoch: 10 , Training Accuracy: 1244/1332 (93%) Training Loss: 0.008888
Validate Accuracy: 289/334 (87%)
Epoch: 11 , Training Accuracy: 1250/1332 (94%) Training Loss: 0.008130
Epoch: 12 , Training Accuracy: 1272/1332 (95%) Training Loss: 0.0077

In [28]:
trainloader, valloader = load_dataloaders(batch_size=16,start=84, end=90)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p02_n079
file p01_n010
file p01_n082
file p01_n107
file p02_n088
file p01_n087
Train start
Epoch: 1 , Training Accuracy: 346/1146 (30%) Training Loss: 0.160455
Epoch: 2 , Training Accuracy: 723/1146 (63%) Training Loss: 0.051595
Epoch: 3 , Training Accuracy: 862/1146 (75%) Training Loss: 0.031025
Epoch: 4 , Training Accuracy: 900/1146 (79%) Training Loss: 0.024598
Epoch: 5 , Training Accuracy: 926/1146 (81%) Training Loss: 0.021893
Validate Accuracy: 222/287 (77%)
Epoch: 6 , Training Accuracy: 923/1146 (81%) Training Loss: 0.021779
Epoch: 7 , Training Accuracy: 936/1146 (82%) Training Loss: 0.020535
Epoch: 8 , Training Accuracy: 952/1146 (83%) Training Loss: 0.019105
Epoch: 9 , Training Accuracy: 955/1146 (83%) Training Loss: 0.017914
Epoch: 10 , Training Accuracy: 949/1146 (83%) Training Loss: 0.017565
Validate Accuracy: 226/287 (79%)
Epoch: 11 , Training Accuracy: 975/1146 (85%) Training Loss: 0.016544
Epoch: 12 , Training Accuracy: 958/1146 (84%) Training Loss: 0.016879
Epoch: 

In [29]:
trainloader, valloader = load_dataloaders(batch_size=16,start=90, end=96)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p05_n086
file p06_n024
file p05_n070
file p04_n016
file p04_n068
file p04_n026
Train start
Epoch: 1 , Training Accuracy: 436/1152 (38%) Training Loss: 0.190598
Epoch: 2 , Training Accuracy: 742/1152 (64%) Training Loss: 0.067578
Epoch: 3 , Training Accuracy: 873/1152 (76%) Training Loss: 0.041641
Epoch: 4 , Training Accuracy: 950/1152 (82%) Training Loss: 0.030883
Epoch: 5 , Training Accuracy: 984/1152 (85%) Training Loss: 0.023764
Validate Accuracy: 242/289 (84%)
Epoch: 6 , Training Accuracy: 1011/1152 (88%) Training Loss: 0.019327
Epoch: 7 , Training Accuracy: 1023/1152 (89%) Training Loss: 0.015743
Epoch: 8 , Training Accuracy: 1035/1152 (90%) Training Loss: 0.014563
Epoch: 9 , Training Accuracy: 1056/1152 (92%) Training Loss: 0.012833
Epoch: 10 , Training Accuracy: 1064/1152 (92%) Training Loss: 0.011613
Validate Accuracy: 256/289 (89%)
Epoch: 11 , Training Accuracy: 1077/1152 (93%) Training Loss: 0.009587
Epoch: 12 , Training Accuracy: 1073/1152 (93%) Training Loss: 0.010015


In [30]:
trainloader, valloader = load_dataloaders(batch_size=16,start=96, end=102)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p06_n039
file p06_n014
file p05_n059
file p02_n051
file p06_n056
file p04_n080
Train start
Epoch: 1 , Training Accuracy: 470/1115 (42%) Training Loss: 0.146659
Epoch: 2 , Training Accuracy: 851/1115 (76%) Training Loss: 0.040801
Epoch: 3 , Training Accuracy: 941/1115 (84%) Training Loss: 0.026040
Epoch: 4 , Training Accuracy: 986/1115 (88%) Training Loss: 0.017838
Epoch: 5 , Training Accuracy: 1006/1115 (90%) Training Loss: 0.013409
Validate Accuracy: 240/279 (86%)
Epoch: 6 , Training Accuracy: 1024/1115 (92%) Training Loss: 0.010900
Epoch: 7 , Training Accuracy: 1034/1115 (93%) Training Loss: 0.010105
Epoch: 8 , Training Accuracy: 1046/1115 (94%) Training Loss: 0.008590
Epoch: 9 , Training Accuracy: 1051/1115 (94%) Training Loss: 0.007171
Epoch: 10 , Training Accuracy: 1062/1115 (95%) Training Loss: 0.006833
Validating Accuracy: 246/279 (88%)
breakout


In [31]:
trainloader, valloader = load_dataloaders(batch_size=16,start=102, end=108)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p05_n062
file p04_n034
file p04_n049
file p06_n033
file p02_n100
file p04_n030
Train start
Epoch: 1 , Training Accuracy: 584/1000 (58%) Training Loss: 0.100868
Epoch: 2 , Training Accuracy: 876/1000 (88%) Training Loss: 0.022717
Epoch: 3 , Training Accuracy: 905/1000 (90%) Training Loss: 0.014816
Epoch: 4 , Training Accuracy: 921/1000 (92%) Training Loss: 0.011331
Epoch: 5 , Training Accuracy: 941/1000 (94%) Training Loss: 0.008742
Validate Accuracy: 230/250 (92%)
Epoch: 6 , Training Accuracy: 949/1000 (95%) Training Loss: 0.007453
Epoch: 7 , Training Accuracy: 946/1000 (95%) Training Loss: 0.006584
Epoch: 8 , Training Accuracy: 959/1000 (96%) Training Loss: 0.006402
Validating Accuracy: 233/250 (93%)
breakout


In [32]:
trainloader = load_testloader(batch_size=16,start=108, end=112)#73 not included 
test(model,device,trainloader)

file p06_n054
file p05_n047
file p06_n021
file p01_n081
Validate Accuracy: 346/1157 (30%)


In [33]:
trainloader= load_vid(batch_size=16,start=1000, end=3000)#73 not included 
evluate(model,device,trainloader)

tensor([ 8, 27, 16,  8,  8,  8,  8, 16,  8,  8,  8,  8,  8, 27,  8, 16],
       device='cuda:0')
tensor([ 8,  8, 16,  8,  8, 16, 16,  8,  8, 16,  8, 16, 27,  8,  8,  8],
       device='cuda:0')
tensor([ 8,  8,  8,  8,  8,  8, 16, 27,  8,  8,  8, 16, 27,  8, 16,  8],
       device='cuda:0')
tensor([27,  8], device='cuda:0')
